In [97]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [98]:
df = pd.read_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/df_2.csv')
df.head()

,Unnamed: 0,price,bed,bath,acre_lot,city,state,house_size,sqft_lot
0,0,105000.0,3.0,2.0,0.12,Adjuntas,Puerto Rico,920.0,5227.2
1,1,80000.0,4.0,2.0,0.08,Adjuntas,Puerto Rico,1527.0,3484.8
2,2,67000.0,2.0,1.0,0.15,Juana Diaz,Puerto Rico,748.0,6534.0
3,3,145000.0,4.0,2.0,0.10,Ponce,Puerto Rico,1800.0,4356.0
4,4,65000.0,6.0,2.0,0.05,Mayaguez,Puerto Rico,1200.0,2178.0


In [99]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    df.drop(['Unnamed: 0', 'acre_lot'], axis=1, inplace=True)  # Droped specified columns
    df['Total_Size'] = df['house_size'] + df['sqft_lot']  # Added new column for total size
    df['Total_Rooms'] = df['bed'] + df['bath'] # Added new columns for total rooms
    grouped_by_city = df.groupby('price',)
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,price,bed,bath,city,state,house_size,sqft_lot,Total_Size,Total_Rooms
0,105000.0,3.0,2.0,Adjuntas,Puerto Rico,920.0,5227.2,6147.2,5.0
1,80000.0,4.0,2.0,Adjuntas,Puerto Rico,1527.0,3484.8,5011.8,6.0
2,67000.0,2.0,1.0,Juana Diaz,Puerto Rico,748.0,6534.0,7282.0,3.0
3,145000.0,4.0,2.0,Ponce,Puerto Rico,1800.0,4356.0,6156.0,6.0
4,65000.0,6.0,2.0,Mayaguez,Puerto Rico,1200.0,2178.0,3378.0,8.0


In [100]:
def kfold_target_encode(df, target_col, cat_col, n_splits=5, smoothing=10):
    encoded_col = pd.Series(index=df.index, dtype=float)
    global_mean = df[target_col].mean()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for train_index, val_index in kf.split(df):
        df_train, df_val = df.iloc[train_index], df.iloc[val_index]
        agg = df_train.groupby(cat_col)[target_col].agg(['count', 'sum'])
        smoothed_mean = (agg['sum'] + global_mean * smoothing) / (agg['count'] + smoothing)
        encoded_col.iloc[val_index] = df_val[cat_col].apply(lambda x: smoothed_mean.get(x, global_mean))
        
    return encoded_col


In [101]:
df_clean['city_encoded'] = kfold_target_encode(df_clean,'price','city')
df_clean['state_encoded'] = kfold_target_encode(df_clean,'price','state')

In [102]:
df = df_clean.copy()

In [103]:
df.drop(index=[0,1,2,3,4,5,6],inplace=True)

In [104]:
# Split the DataFrame into a list of 4 DataFrames
list_of_dfs = np.array_split(df, 4)

# Assign each part to its own new DataFrame variable
df1 = list_of_dfs[0]
df2 = list_of_dfs[1]
df3 = list_of_dfs[2]
df4 = list_of_dfs[3]
df_clean.to_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df_clean.csv')
df1.to_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df1.csv')
df2.to_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df2.csv')
df3.to_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df3.csv')
df4.to_csv('E:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Cleaned_Data/Split_Data/df4.csv')

# --- Verification ---
print("/nShapes of the 4 new DataFrames:")
print(f"df1 shape: {df1.shape}")
print(f"df2 shape: {df2.shape}")
print(f"df3 shape: {df3.shape}")
print(f"df4 shape: {df4.shape}")

e:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\Prediction_Model\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


/nShapes of the 4 new DataFrames:
df1 shape: (556240, 11)
df2 shape: (556240, 11)
df3 shape: (556240, 11)
df4 shape: (556240, 11)
